In [1]:
import inspect
import sys
import types

In [2]:
types.FunctionType, types.MethodType

(function, method)

In [3]:
class Test:
    pass

OrigTest = Test
TOBJ = Test()

print(Test,id(Test),TOBJ,id(TOBJ))

class Bar:
    def imeth(self,*args):
        return 'imeth',self,args
    @staticmethod
    def smeth(self,*args):
        return 'smeth',self,args
    @classmethod
    def cmeth(self,*args):
        return 'cmeth',self,args

<class '__main__.Test'> 35256392 <__main__.Test object at 0x7f3be16fd8d0> 139895162001616


In [4]:
for k,v in inspect.getmembers(Bar):
    if not k.startswith('__'):
        print(k,type(v),v,id(v))
print()
for k,v in Bar.__dict__.items():
    if not k.startswith('__'):
        print(k,type(v),v,id(v))

cmeth <class 'method'> <bound method Bar.cmeth of <class '__main__.Bar'>> 139895179786760
imeth <class 'function'> <function Bar.imeth at 0x7f3be1f7b6a8> 139895170905768
smeth <class 'function'> <function Bar.smeth at 0x7f3be1f7b730> 139895170905904

cmeth <class 'classmethod'> <classmethod object at 0x7f3be16fdbe0> 139895162002400
imeth <class 'function'> <function Bar.imeth at 0x7f3be1f7b6a8> 139895170905768
smeth <class 'staticmethod'> <staticmethod object at 0x7f3be16fd940> 139895162001728


In [5]:
def extend(newcls):
    clsname = newcls.__name__
    oldcls = getattr(sys.modules[newcls.__module__],clsname,None)
    if oldcls is None:
        raise TypeError("Class '{}' not previously defined; therefore cannot be extended."
                        .format(clsname))

    if len(newcls.__bases__) > 1 or (len(newcls.__bases__) == 1 and newcls.__bases__[0] is not object):
        raise TypeError("Class '{}' extension cannot have any base classes."
                        .format(clsname))

    for a,v in newcls.__dict__.items():
        if not a.startswith('__'):
            ## print(a,v,type(v))
            if hasattr(oldcls,a):
                raise TypeError("Attribute '{}' in extended class '{}' already exists. Cannot be redefined."
                               .format(a,clsname))
            setattr(oldcls,a,v)
            
    return oldcls

In [6]:
@extend
class Test:
    A = 1

In [7]:
Test, id(Test)

(__main__.Test, 35256392)

In [8]:
TOBJ.A

1

In [9]:
Test is OrigTest

True

In [10]:
@extend
class Test:
    def imeth(self,*args):
        return self,args

In [11]:
Test is OrigTest

True

In [12]:
TOBJ.imeth(10,20,30), TOBJ.imeth(10)[0].__class__ is OrigTest

((<__main__.Test at 0x7f3be16fd8d0>, (10, 20, 30)), True)

In [13]:
t2 = Test()
t2.imeth(30,40,50), t2.imeth()[0].__class__ is OrigTest

((<__main__.Test at 0x7f3be170bd30>, (30, 40, 50)), True)

In [14]:
@extend
class Test:
    @classmethod
    def cmeth(self,*args):
        return self,args

In [15]:
Test is OrigTest

True

In [16]:
TOBJ.cmeth(10,20,30,40), TOBJ.cmeth(10)[0] is OrigTest

((__main__.Test, (10, 20, 30, 40)), True)

In [17]:
t3 = Test()
t3.cmeth(40,50),t3.imeth(1),t3.A, t3.cmeth()[0] is OrigTest

((__main__.Test, (40, 50)), (<__main__.Test at 0x7f3be170b9b0>, (1,)), 1, True)

In [18]:
@extend
class Test:
    @staticmethod
    def smeth(self,*args):
        return self,args

In [19]:
Test is OrigTest

True

In [20]:
TOBJ.smeth(1)

(1, ())

In [21]:
t4 = Test()
t4.A, t4.imeth(100), t4.cmeth(200), t4.smeth(300), \
t4.imeth()[0].__class__ is OrigTest, t4.cmeth()[0] is OrigTest

(1,
 (<__main__.Test at 0x7f3be17096d8>, (100,)),
 (__main__.Test, (200,)),
 (300, ()),
 True,
 True)

In [22]:
@extend
class Test:
    
    @property
    def P(self):
        return self.pp
    
    @P.setter
    def P(self,v):
        self.pp = v*2
    
    @P.deleter
    def P(self):
        del self.pp


In [23]:
TOBJ.P = 10
TOBJ.P

20

In [24]:
del TOBJ.P

In [25]:
try:
    TOBJ.P
except Exception as e:
    print(e.__class__,e)

<class 'AttributeError'> 'Test' object has no attribute 'pp'


In [26]:
t5 = Test()
t5.P = 1
t5.P, t5.smeth(10), t5.cmeth(20,30), t5.imeth(40,50,60)

(2,
 (10, ()),
 (__main__.Test, (20, 30)),
 (<__main__.Test at 0x7f3be171e278>, (40, 50, 60)))

In [27]:
try:
    @extend
    class Test:
        def imeth(self):
            pass
except Exception as e:
    print(e.__class__,e)

<class 'TypeError'> Attribute 'imeth' in extended class 'Test' already exists. Cannot be redefined.


In [28]:
try:
    @extend
    class Test(Bar):
        def imeth(self):
            pass
except Exception as e:
    print(e.__class__,e)

<class 'TypeError'> Class 'Test' extension cannot have any base classes.
